In [2]:
import requests
from bs4 import BeautifulSoup as bs
import re
import csv
 
import pandas as pd

url = 'https://www.rockefellerfoundation.org/grants/'

#requesting a session in the specific url
with requests.Session() as s:
    r = s.get(url)
    src = r.content
    soup = bs(src)
    soup.prettify()
    descriptions = []
    allPages = []
    amounts = []
    names = []
    while url != '':  
        #print(url)
        r = s.get(url)
        src = r.content
        soup = bs(src)
        soup.prettify()
        next_link = str(soup.find_all('li', {'class': 'next btn'}))
        next_link = next_link[60:]
        next_link = next_link[:next_link.find('>')]
        data = soup.find_all('div', {'class': 'grant-left'})
        for link in data:
                url = link.find('a').get('href')
                allPages.append(url)
                page_source = s.get(url).content
                soup = bs(page_source) 
                soup.prettify()
                soup = str(soup)
                description_index1 = soup.find("<h2>Description:</h2> <span>")
                description_index2 = soup[description_index1:].find("</sp")
                descriptions.append(soup[description_index1+28:description_index1 + description_index2])
                name_index1 = soup.find("<title>")
                name_index2 = soup[name_index1:].find("-") - 1
                names.append(soup[name_index1+7:name_index1 + name_index2])
                amount_index1 = soup.find("<span>$")
                amount_index2 = soup[amount_index1:].find("</span>")
                amounts.append(soup[amount_index1 + 6:amount_index1  + amount_index2])
        url = next_link 
    
    
DataFrame = pd.DataFrame()
DataFrame['Page URL'] = allPages
DataFrame['Name'] = names
DataFrame['Description'] = descriptions
DataFrame['Amount'] = amounts
    
Data = DataFrame.drop_duplicates() 
Data.to_csv("Rockefeller_Funding_Data.csv")

print('Webscraping complete')

Webscraping complete
